In [46]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [47]:
datos = pd.read_excel(r"C:\Users\dapen\OneDrive - Universidad de los andes\MIAD\Visualización y Storytelling\Proyecto\Datos.xlsx",
                      sheet_name="Export")

## Transformación de datos

# Obtener columna Fecha_produccion: Fecha_vencimiento  - Vida_util
datos["Vida_util_timedelta"] = pd.to_timedelta(datos["Vida_util"], unit='D')
datos["Fecha_produccion"] = datos["Fecha_vencimiento"] - datos["Vida_util_timedelta"]

# Split por espacio de la columna Referencia para obtener Marca, Empaque, Volumen, Und_volumen, Retornable y Texto
datos[["Marca", "Empaque", "Volumen", "Und_Volumen", "Retornable","Texto"]] = datos["Referencia"].str.split(expand=True)

# Cambiar Retornable por Si
datos["Retornable"] = datos["Retornable"].replace("Retornable", "SI")

# Creación de columnas año_produccion y mes_produccion, a partir de Fecha_produccion
datos['año_produccion'] = datos['Fecha_produccion'].dt.year
datos['mes_produccion'] = datos['Fecha_produccion'].dt.month

# Creación de columnas año_encuesta y mes_encuesta, a partir de Fecha_encuesta
datos['año_encuesta'] = datos['Fecha_encuesta'].dt.year
datos['mes_encuesta'] = datos['Fecha_encuesta'].dt.month

# Categorización por frescura (Ideal, debe mejorar a inaceptable)
meses = {
    1: 'Ene', 2: 'Feb', 3: 'Mar',
    4: 'Abr', 5: 'May', 6: 'Jun',
    7: 'Jul', 8: 'Ago', 9: 'Sept',
    10: 'Oct', 11: 'Nov', 12: 'Dic'
}

datos['mes_encuesta_cat'] = datos["mes_encuesta"].map(meses)
datos['mes_produccion_cat'] = datos["mes_produccion"].map(meses)

# Eliminación de columnas innecesarias para la visualización
datos = datos.drop(columns=["Referencia", "Establecimiento", "Lote", 
                            "Vida_util_timedelta", "Texto", "Und_Volumen", 
                            "Fecha_vencimiento", "mes_encuesta", "mes_produccion"])

In [48]:
# Función para clasificar la frescura
def calculo_frescura(row):
    # Condiciones Marca_1, Marca_2, Marca_3
    if row['Marca'] in ['Marca_1', 'Marca_2', 'Marca_3']:
        if row['Edad_producto'] < 90:
            return 'Ideal'
        elif 90 < row['Edad_producto'] <= 120:
            return 'Debe Mejorar'
        else:
            return 'Inaceptable'
    # Condiciones Marca_4, Marca_5, Marca_6, Marca_7
    elif row['Marca'] in ['Marca_4', 'Marca_5', 'Marca_6', 'Marca_7']:
        if row['Edad_producto'] < 60:
            return 'Ideal'
        elif 60 < row['Edad_producto'] <= 100:
            return 'Debe Mejorar'
        else:
            return 'Inaceptable'

# Aplicar la función para crear la nueva columna
datos['Frescura'] = datos.apply(calculo_frescura, axis=1)

In [51]:
meses_ordenados = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']
datos['mes_encuesta_cat'] = pd.Categorical(datos['mes_encuesta_cat'], categories=meses_ordenados, ordered=True)

# Crear un gráfico de cajas agrupado por mes
fig = px.box(datos, x='mes_encuesta_cat', y='Edad_producto', title='Distribución de Valores por Mes')

# Mostrar el gráfico
fig.show()

In [52]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13268 entries, 0 to 13267
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Ciudad              13268 non-null  object        
 1   Canal               13268 non-null  object        
 2   Edad_producto       13268 non-null  int64         
 3   Vida_util           13268 non-null  int64         
 4   Fecha_encuesta      13268 non-null  datetime64[ns]
 5   Fecha_produccion    13268 non-null  datetime64[ns]
 6   Marca               13268 non-null  object        
 7   Empaque             13268 non-null  object        
 8   Volumen             13268 non-null  object        
 9   Retornable          13268 non-null  object        
 10  año_produccion      13268 non-null  int32         
 11  año_encuesta        13268 non-null  int32         
 12  mes_encuesta_cat    12173 non-null  category      
 13  mes_produccion_cat  13268 non-null  object    